In [2]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 48.5 MB/s eta 0:00:00


In [4]:
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value


nutrient_contributions = np.array([
    [3, 1, 1],  # Product 1
    [1, 2, 0],  # Product 2
    [1, 6, 0],  # Product 3
])
nutrient_requirements = np.array([9, 8, 12])
product_costs = np.array([5, 6, 0])


def validate_inputs(contributions, requirements, costs):
    if not all(isinstance(arr, np.ndarray) for arr in [contributions, requirements, costs]):
        raise TypeError("All inputs must be numpy arrays.")
    if contributions.ndim != 2 or requirements.ndim != 1 or costs.ndim != 1:
        raise ValueError("Ensure contributions is 2D and requirements/costs are 1D.")
    if contributions.shape[0] != requirements.size or contributions.shape[1] != costs.size:
        raise ValueError("Mismatch between input dimensions.")
    if any(arr.min() < 0 for arr in [requirements, costs, contributions.flatten()]):
        raise ValueError("All input values must be non-negative.")
    if not all(np.isfinite(arr).all() for arr in [contributions, requirements, costs]):
        raise ValueError("All input values must be finite.")

validate_inputs(nutrient_contributions, nutrient_requirements, product_costs)


problem = LpProblem("Nutrient_Optimization", LpMinimize)


product_quantities = [
    LpVariable(f"Product_{i+1}", lowBound=0, cat="Continuous")
    for i in range(len(product_costs))
]


problem += lpSum(product_costs[i] * product_quantities[i] for i in range(len(product_costs))), "Total_Cost"


for idx, requirement in enumerate(nutrient_requirements):
    problem += (
        lpSum(nutrient_contributions[idx, i] * product_quantities[i] for i in range(len(product_costs))) >= requirement,
        f"Nutrient_{idx+1}_Requirement"
    )


problem.solve()


if LpStatus[problem.status] == "Optimal":
    print("Optimal solution determined:")
    for var in product_quantities:
        print(f"Variable {var.name}: {var.varValue:.2f}")
    print(f"Objective Function Value (Total Cost): {value(problem.objective):.2f}")
else:
    print("Optimal solution not attained. Further analysis is required.")



Optimal solution determined:
Variable Product_1: 0.00
Variable Product_2: 4.00
Variable Product_3: 5.00
Objective Function Value (Total Cost): 24.00


In [ ]:
# Value type: choose between 'Continuous' or 'Integer'
value_type_options = {'Continuous', 'Integer'};
value_type = value_type_options{1};  # Change this to {2} for integer variables

# Data
nutrients_per_unit = [
    1, 5;    # S1
    3, 2;    # S2
    2, 4;    # S3
    2, 2;    # S4
    1, 0     # S5
];

max_nutrients = [10; 12; 16; 10; 1];
costs = [2; 3];

# Number of products and nutrients
[num_nutrients, num_products] = size(nutrients_per_unit);

# Define the coefficients for the objective function (costs)
c = costs;

# Define the constraint matrix (nutrients_per_unit) and the RHS (max_nutrients)
A = nutrients_per_unit;
b = max_nutrients;

# Lower bounds for decision variables (x >= 0)
lb = zeros(num_products, 1);

# Upper bounds (leave it unbounded by setting it to an empty matrix)
ub = [];

# We want to minimize the objective function
sense = 1;  # 1 = minimization

# Define CTYPE: character vector to specify type of each constraint
ctype = repmat('L', num_nutrients, 1);  # All constraints are lower bounds (Ax >= b)

# Define VARTYPE based on value_type (continuous or integer)
if strcmp(value_type, 'Continuous')
    vartype = repmat('C', num_products, 1);  # Continuous variables
else
    vartype = repmat('I', num_products, 1);  # Integer variables
end

# Solve the linear programming problem using glpk
[x, fval, exitflag] = glpk(c, A, b, lb, ub, ctype, vartype, sense);

# Display the results
if exitflag == 0
    disp('Status: Optimal solution found');
    for i = 1:num_products
        fprintf('x%d = %.4f\n', i, x(i));
    end
    fprintf('Total Cost of the diet: %.4f\n', fval);
else
    disp('Problem not solved');
end
